In [0]:
# The SparkContext/SparkSession is the entry point for all Spark operations
# sc = the SparkContext = the execution environment of Spark, only 1 per JVM
# Note that SparkSession is now the entry point (from Spark v2.0)
# This tutorial uses SparkContext (was used prior to Spark 2.0)

from pyspark import SparkContext
# sc = SparkContext(appName = "some_application_name") # You'd normally run this, but in this case, it has already been created in the Databricks' environment

In [0]:
quote = "To be, or not to be, that is the question: Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune, Or to take Arms against a Sea of troubles, And by opposing end them: to die, to sleep No more; and by a sleep, to say we end the heart-ache, and the thousand natural shocks that Flesh is heir to? 'Tis a consummation devoutly to be wished. To die, to sleep, To sleep, perchance to Dream; aye, there's the rub, for in that sleep of death, what dreams may come, when we have shuffled off this mortal coil, must give us pause."

https://spark.apache.org/docs/2.1.1/programming-guide.html#parallelized-collections

In [0]:
sparkdata = sc.parallelize(quote.split(' '))

In [0]:
sparkdata

Out[4]: ParallelCollectionRDD[89] at readRDDFromInputStream at PythonRDD.scala:413

In [0]:
print ("sparkdata = ", sparkdata)
print ("sparkdata.collect() = ", sparkdata.collect()[1:10])

sparkdata = ParallelCollectionRDD[89] at readRDDFromInputStream at PythonRDD.scala:413
sparkdata.collect() = ['be,', 'or', 'not', 'to', 'be,', 'that', 'is', 'the', 'question:']

In [0]:
# A simple transformation - map
def mapword(word):
  return (word,1)

In [0]:
print (sparkdata.map(mapword) )# Nothing has happened here
print (sparkdata.map(mapword).collect()[1:10]) # collect causes the DAG to execute

PythonRDD[90] at RDD at PythonRDD.scala:58
[('be,', 1), ('or', 1), ('not', 1), ('to', 1), ('be,', 1), ('that', 1), ('is', 1), ('the', 1), ('question:', 1)]

In [0]:
wordCountsCollected = (sparkdata
 .map(mapword)
 .reduceByKey(lambda a,b: a+b)
 .collect())

In [0]:
print (wordCountsCollected)

[('of', 3), ('Or', 1), ('against', 1), ('troubles,', 1), ('And', 1), ('aye,', 1), ('rub,', 1), ('when', 1), ('that', 3), ('Whether', 1), ('outrageous', 1), ('a', 3), ('by', 2), ('die,', 2), ('No', 1), ('consummation', 1), ('take', 1), ('end', 2), ('we', 2), ('heir', 1), ('death,', 1), ('this', 1), ('question:', 1), ('fortune,', 1), ('Arms', 1), ('opposing', 1), ('natural', 1), ('for', 1), ('what', 1), ('dreams', 1), ('shuffled', 1), ("'tis", 1), ('in', 2), ('mind', 1), ('arrows', 1), ('say', 1), ('thousand', 1), ('to?', 1), ('devoutly', 1), ('wished.', 1), ('Dream;', 1), ('may', 1), ('come,', 1), ('have', 1), ('must', 1), ('give', 1), ('us', 1), ('be,', 2), ('to', 9), ('the', 5), ('slings', 1), ('and', 3), ('sleep', 2), ('heart-ache,', 1), ('shocks', 1), ('Flesh', 1), ('be', 1), ('perchance', 1), ('off', 1), ('coil,', 1), ('is', 2), ('nobler', 1), ('suffer', 1), ('The', 1), ('Sea', 1), ('more;', 1), ('sleep,', 3), ("there's", 1), ('mortal', 1), ('pause.', 1), ('To', 3), ('or', 1), ('not', 1), ('them:', 1), ("'Tis", 1)]

In [0]:
ДЗ1:
  Написать функцию для Map, которая вернет все символы длинее 2.

File "<command-3033100966802525>" , line 1 
 ДЗ1: 
 ^
 SyntaxError : invalid syntax

In [0]:
def charsmorethan2(tuple1):
  if len(tuple1[0]) > 2:
    return tuple1

In [0]:
rdd3 = sparkdata.map(mapword).filter(lambda x: charsmorethan2(x))
# Multiple Transformations in 1 statement, nothing is happening yet
rdd3.collect()[1:10] 
# The DAG gets executed. Note that since we didn't remove punctuation marks ... 'be,', etc are also included

Out[21]: [('not', 1),
 ('be,', 1),
 ('that', 1),
 ('the', 1),
 ('question:', 1),
 ('Whether', 1),
 ("'tis", 1),
 ('nobler', 1),
 ('the', 1)]

In [0]:
# With Tables, a general example
cms = sc.parallelize([[1,"Dr. A",12.50,"Yale"],[2,"Dr. B",5.10,"Duke"],[3,"Dr. C",200.34,"Mt. Sinai"],[4,"Dr. D",5.67,"Duke"],[1,"Dr. E",52.50,"Yale"]])

In [0]:
def findPayment(data):
  return data[2]

In [0]:
print ("Payments = ", cms.map(findPayment).collect())
print ("Mean = ", cms.map(findPayment).mean() )# Mean is an action

Payments = [12.5, 5.1, 200.34, 5.67, 52.5]
Mean = 55.222

ДЗ2: 
  В таблице выше найдите  стандартное отклонение, минимальное, максимальное значения, сумму всех выплат.

In [0]:
print ("Payments = ", cms.map(findPayment).collect())
print ('Standard Deviation = ', cms.map(findPayment).stdev())
print ('Minimal Value = ', cms.map(findPayment).min())
print ('Maximum Value = ', cms.map(findPayment).max())
print ('Total Sum = ', cms.map(findPayment).sum())

Payments = [12.5, 5.1, 200.34, 5.67, 52.5]
Standard Deviation = 74.64506424406103
Minimal Value = 5.1
Maximum Value = 200.34
Total Sum = 276.11

In [0]:
import pyspark.sql.functions as func

# Creating a DataFrame (familiar to Python programmers)
cms_df = sqlContext.createDataFrame(cms, ["ID","Name","Payment","Hosp"])
print (cms_df.show())
print (cms_df.groupby('Hosp').agg(func.avg('Payment'), func.max('Payment'),func.min('Payment')))
print (cms_df.groupby('Hosp').agg(func.avg('Payment'), func.max('Payment'),func.min('Payment')).collect())
print()
print( "Converting to a Pandas DataFrame")
print( "--------------------------------")
pd_df = (cms_df.groupby('Hosp').agg(func.avg('Payment'), func.max('Payment'),func.min('Payment')).toPandas())
print (type(pd_df))
print()
print (pd_df)


+---+-----+-------+---------+
 ID| Name|Payment| Hosp|
+---+-----+-------+---------+
 1|Dr. A| 12.5| Yale|
 2|Dr. B| 5.1| Duke|
 3|Dr. C| 200.34|Mt. Sinai|
 4|Dr. D| 5.67| Duke|
 1|Dr. E| 52.5| Yale|
+---+-----+-------+---------+

None
DataFrame[Hosp: string, avg(Payment): double, max(Payment): double, min(Payment): double]
[Row(Hosp='Yale', avg(Payment)=32.5, max(Payment)=52.5, min(Payment)=12.5), Row(Hosp='Duke', avg(Payment)=5.385, max(Payment)=5.67, min(Payment)=5.1), Row(Hosp='Mt. Sinai', avg(Payment)=200.34, max(Payment)=200.34, min(Payment)=200.34)]

Converting to a Pandas DataFrame
--------------------------------
<class 'pandas.core.frame.DataFrame'>

 Hosp avg(Payment) max(Payment) min(Payment)
0 Yale 32.500 52.50 12.50
1 Duke 5.385 5.67 5.10
2 Mt. Sinai 200.340 200.34 200.34

In [0]:
wordsList = ['to','be','or','not','to','be']
wordsRDD = sc.parallelize(wordsList, 3) # Splits into 2 groups
# Print out the type of wordsRDD
print (type(wordsRDD))

<class 'pyspark.rdd.RDD'>

In [0]:
wordsRDD.collect()

Out[28]: ['to', 'be', 'or', 'not', 'to', 'be']

In [0]:
# An example with changing the case of words
# One way of completing the function
def makeUpperCase(word):
  return word.upper()

print (makeUpperCase('cat'))

CAT

In [0]:
upperRDD = wordsRDD.map(makeUpperCase)
print( upperRDD.collect())


['TO', 'BE', 'OR', 'NOT', 'TO', 'BE']

In [0]:
upperLambdaRDD = wordsRDD.map(lambda word: word.upper())
print (upperLambdaRDD.collect())

['TO', 'BE', 'OR', 'NOT', 'TO', 'BE']